<a href="https://colab.research.google.com/github/micah-shull/LLMs/blob/main/LLM_006_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gradio - What is it?

Gradio is an open-source Python library that allows you to quickly create and deploy user interfaces (UIs) for machine learning models, APIs, or any interactive applications. It is particularly useful for creating simple web interfaces where users can interact with models in real time, such as uploading images, entering text, or recording audio, and then receiving outputs directly on the web page.

What makes Gradio special:
- **Ease of use**: You can create a fully functional interface with just a few lines of Python code.
- **Real-time interaction**: It enables users to interact with models and see results instantly, which is especially useful for testing models like LLMs, image classifiers, or translators.
- **Customizable interfaces**: Gradio supports a wide range of input and output types (text, images, video, audio, etc.), making it flexible for various use cases.
- **Shareability**: Gradio automatically generates a shareable link, allowing you to easily share your model's interface with others.
- **Deployment**: It is widely used for demos, prototyping, or even full-scale deployment, and integrates smoothly with models from frameworks like PyTorch, TensorFlow, Hugging Face, etc.

Gradio is especially appealing for rapid prototyping and creating accessible tools for machine learning applications.

### Install Libraries

In [3]:
# !pip install python-dotenv
# !pip install openai
# !pip install google-generativeai
# !pip install anthropic
# !pip install gradio

### Import Libraries

In [4]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

### Write API Keys to .env file

In [5]:
# Path to the .env file
env_file_path = '/content/API_KEYS.env' # naming file makes it visible, no name makes it hidden

# Your OpenAI API key
OPEN_API_KEY = "sk-proj-e1GUWru####"
ANTHROPIC_API_KEY = "sk-ant-api####"
GOOGLE_API_KEY = "AIzaSyDh3aiDL####"

# Create the .env file and write the API keys
with open(env_file_path, 'w') as f:
    f.write(f"OPENAI_API_KEY={OPEN_API_KEY}\n")
    f.write(f"ANTHROPIC_API_KEY={ANTHROPIC_API_KEY}\n")
    f.write(f"GOOGLE_API_KEY={GOOGLE_API_KEY}\n")

print(f".env file created at: {env_file_path}")
# List all files (including hidden ones) in the /content/ folder
!ls -la /content/

.env file created at: /content/API_KEYS.env
total 20
drwxr-xr-x 1 root root 4096 Oct 25 18:39 .
drwxr-xr-x 1 root root 4096 Oct 25 18:36 ..
-rw-r--r-- 1 root root  362 Oct 25 18:39 API_KEYS.env
drwxr-xr-x 4 root root 4096 Oct 24 13:20 .config
drwxr-xr-x 1 root root 4096 Oct 24 13:20 sample_data


### Load Environment Variables

Let's clarify how `os.getenv()` and environment variables work to clear up any confusion.

1. **What `os.getenv()` does**:
   - `os.getenv('key')` retrieves the value of the environment variable named `'key'`. It does not store values; it simply **fetches** the value of an environment variable. Each time you call `os.getenv()`, it fetches the value of the specific environment variable you ask for.
   - For example:
     ```python
     openai_key = os.getenv('OPENAI_API_KEY')  # Fetches the OpenAI API key
     google_key = os.getenv('GOOGLE_API_KEY')  # Fetches the Google API key
     ```
     Each call retrieves a **different** environment variable. These are independent of each other.


This approach is better because:
1. **It avoids redundancy**: You don't need to manually set `os.environ` after loading the variables with `os.getenv()`.
2. **Direct use of `os.getenv()`**: It is cleaner and ensures that you fetch the API key values directly from the environment.
3. **Ensures proper error handling**: You can check if each key is loaded correctly and raise an error or alert if something is missing.

### When should you use the alternative (`os.environ` method)?
You would typically set `os.environ` manually only if you need the environment variables to be explicitly available in a subprocess or for other libraries that expect the keys in the environment (rare cases).


In [6]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv('/content/API_KEYS.env')  # Ensure this is the correct path to your file

# Get the API keys from the environment
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

# Check if the keys are loaded correctly and print a portion of them
if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[0:10]}...")  # Only print part of the key
else:
    print("OpenAI API key not loaded correctly.")

if anthropic_api_key:
    print(f"Anthropic API Key loaded: {anthropic_api_key[0:10]}...")
else:
    print("Anthropic API key not loaded correctly.")

if google_api_key:
    print(f"Google API Key loaded: {google_api_key[0:10]}...")
else:
    print("Google API key not loaded correctly.")


OpenAI API Key loaded: sk-proj-e1...
Anthropic API Key loaded: sk-ant-api...
Google API Key loaded: AIzaSyDh3a...


### Connect to OpenAI, Anthropic and Google

This code block is configuring connections to three different large language model (LLM) services: OpenAI, Anthropic (Claude), and Google Generative AI. By setting the appropriate API keys, it authenticates your script with each service, enabling you to send requests to these platforms and use their models for tasks like text generation, answering questions, or other AI-driven functionalities. Once connected, you can interact with each platform's models securely and programmatically within your script.

In [7]:
import openai
import anthropic
import google.generativeai

# Connect to OpenAI
openai.api_key = openai_api_key  # Set OpenAI API key

# Connect to Anthropic (Claude)
claude = anthropic.Anthropic(api_key=anthropic_api_key)  # Set Anthropic API key

# Connect to Google Generative AI
google.generativeai.configure(api_key=google_api_key)  # Set Google API key

## Function Call to GPT-4o-Mini

This function, `message_gpt()`, sends a user’s prompt to OpenAI’s GPT-4 model (specifically `'gpt-4o-mini'`), along with a predefined system message (`"You are a helpful assistant"`), and returns the model's response.

### Summary of what it does:
1. **System message**: The system message establishes the behavior of the AI as a "helpful assistant."
2. **User input**: The user's prompt is passed to the function.
3. **API call**: The function sends both the system message and the user prompt to OpenAI’s API (for GPT-4), specifying the model to use.
4. **Response**: It retrieves the first response from the model and returns it.

In short, it wraps an API call to GPT-4, allowing the user to send a prompt and get a response based on the assistant-like behavior defined by the system message.

In [14]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

message_gpt("What is today's date?") # last day of training for gpt-4o-mini

"Today's date is October 3, 2023."

In [13]:
from datetime import date

today = date.today()
print("Today's actual date:", today)

Today's actual date: 2024-10-25


## Gradio Introduction

In [15]:
# here's a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

shout("hello")

Shout has been called with input hello


'HELLO'

### Gradio User Interface

In [16]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://03a35f0232567251e4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Increase the Size of Textbox

In [20]:
view = gr.Interface(
    fn=shout,# still using the shout function
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    allow_flagging="never" # removes flag
)
view.launch(share=True, debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0ea9356368fd032ad5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Shout has been called with input testing out the debugger
Shout has been called with input testing out the debugger to show the output in the cell below
Shout has been called with input testing out the debugger to show the output in the cell below for a third time
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://03a35f0232567251e4.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://36255b38bbe79f2df3.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://28b45d3e6ee9cd1450.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://32937b94d19d3facfc.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://0ea9356368fd032ad5.gradio.live


## Gradio & ChatGPT (Chatbot)

The key concepts and lessons to take away from this code are how to combine ChatGPT with a Gradio interface to create a simple interactive chatbot, specifically focusing on system and user messages in ChatGPT and how Gradio provides an easy interface for it.

### Major Concepts You Should Be Learning:

1. **System and User Messages in ChatGPT**:
   - **System Messages**: These set the tone or behavior of the assistant (in this case, "You are a helpful assistant"). This message is sent as a context to the model to influence its behavior throughout the conversation.
   - **User Messages**: These are the actual inputs or prompts from the user, which the model responds to. In this function, the user's message is included as `{"role": "user", "content": prompt}` in the message payload.
   - **Message Structure**: You're learning how to build a structured conversation with ChatGPT using roles (system, user) to define different parts of the interaction.

2. **Gradio**:
   - **What Gradio Does**: Gradio creates an easy-to-use web-based interface to interact with machine learning models. In this case, it is used to create a simple text-based chatbot interface.
   - **Major Components**:
     - `fn=message_gpt`: This tells Gradio to use your `message_gpt()` function to handle the user input.
     - `inputs=[gr.Textbox(...)]`: This creates a text input box where the user enters their message (or prompt).
     - `outputs=[gr.Textbox(...)]`: This defines where the AI’s response will be displayed.
     - `view.launch(share=True)`: This starts the Gradio interface, and setting `share=True` allows you to share the app via a unique URL that Gradio generates.

3. **Combining ChatGPT with Gradio**:
   - **How It Works**: You're learning to connect ChatGPT's API (via the `message_gpt()` function) to a web interface using Gradio. When a user types something in the input box, Gradio sends it to the `message_gpt()` function, which sends the message to ChatGPT and returns the response to be displayed in the output box.
   - **User Interaction**: The Gradio interface captures user input and displays the AI's response, providing an easy-to-use web interface without having to write complex frontend code.

### Major Lessons:
1. **System messages control model behavior**: You can adjust the system message to define the "personality" or behavior of the assistant (helpful, formal, humorous, etc.).
2. **Structured communication with the model**: You're building a structured conversation with roles (system and user) and learning how ChatGPT uses this to generate context-aware responses.
3. **Gradio simplifies UI creation**: Gradio is a powerful tool for quickly creating web interfaces for machine learning models, allowing you to easily deploy and share models with others without needing to build a complex interface from scratch.

This code teaches how to structure a basic conversational interaction with ChatGPT, connect it to a Gradio interface, and launch it as an interactive application.

### Format Response in Markdown

In [30]:
# use our function from earlier

system_message = "You are a helpful assistant"

# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

# add GPT to our Gradio interface
view = gr.Interface(
    fn=message_gpt, # now we use GPT
    inputs=[gr.Textbox(label="Your message:", lines=4)],
    # outputs=[gr.Textbox(label="Response:", lines=8)], # respond in plain text
    outputs=[gr.Markdown(label="Response:")], # add markdown formatting, # add markdown formatting
    flagging_mode="never"
)
view.launch(share=True, debug=True) # debug=True keeps the cell running ubtil stopped manually


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://50a882f6d55240eb47.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7864 <> https://50a882f6d55240eb47.gradio.live


### Stream Response

This code introduces a new concept: **streaming responses from ChatGPT**. Here's a breakdown of the major lessons and concepts you should focus on when learning from this code:

### Major Concepts You Should Be Learning:

1. **Streaming Responses from ChatGPT**:
   - **What is Streaming?**: Instead of waiting for the entire response to be generated before returning it, this code **streams** the response in real time. This allows the user to see parts of the response as they are being generated, providing faster feedback and a smoother user experience.
   - **How Streaming Works**:
     - `stream=True`: This enables streaming mode in the ChatGPT API. Instead of returning the entire message all at once, the API sends smaller chunks of the response as they are generated.
     - **Chunking the Response**: The response is streamed in **chunks**, which are processed one by one in the loop. Each `chunk` is a part of the overall message being constructed.
   
2. **The Role of `yield`**:
   - **What `yield` Does**: Instead of returning the full response at once, `yield` returns a partial result (the accumulated response so far) each time a new chunk is received. This allows for real-time feedback in the interface.
   - **Building the Response**: The `result` variable is used to accumulate the chunks of the response. Each new chunk of text is appended to the previous ones, and `yield result` sends this cumulative text back to the user interface (or caller) as it is updated.

3. **System and User Messages**:
   - As with the previous examples, this function still uses **system** and **user messages** to structure the interaction. The **system message** defines the assistant's behavior, while the **user message** contains the prompt.

4. **Combining Streaming with the User Interface**:
   - You’re learning a key concept here: how to stream AI responses back to the user in real time, which enhances the user experience. When incorporated with Gradio or any web interface, this creates a **live, dynamic interaction** where users don’t have to wait for the entire response to be generated before seeing any output.

### Major Lessons:

1. **Streaming Responses Improves User Experience**:
   - Streaming can provide faster feedback, particularly useful for longer responses where you want users to see results as they are generated.
2. **Yielding Responses Allows Incremental Output**:
   - The use of `yield` makes the function return intermediate results, allowing the UI (like Gradio) to update in real time, providing a more dynamic user experience.
3. **Streaming Mode in OpenAI API**:
   - You're learning how to activate and manage **streaming** in OpenAI's API by setting `stream=True`. This can be particularly useful for chatbots, customer support, or any application where response speed is important.

### Practical Application:
- In the context of Gradio, this function can be integrated to show the response being typed out in real time as users interact with the model, improving the responsiveness of the chatbot interface.


### Delta

The `delta` in the code refers to the incremental updates (or partial content) that are streamed from the OpenAI API when `stream=True` is enabled.

In streaming mode, the API sends parts of the response as they are generated, rather than waiting to return the full response all at once. Each part, or "chunk," is received as a message containing the **difference** (or **delta**) between the last part of the response and the current part.

### Key Concepts:
1. **Delta**:
   - The term "delta" comes from mathematics, where it represents the change or difference between values. In this case, `delta` represents the difference between the response's current state and the next chunk of content generated.
   - The OpenAI API uses `delta` to represent the partial text being added to the ongoing response. Each new `chunk` contains only the newly generated text that is being streamed at that moment.

2. **How it Works**:
   - In streaming mode, the API sends multiple "chunks" of data, and each chunk contains the new text that was generated since the previous chunk.
   - `chunk.choices[0].delta.content` accesses the newly generated text from the `delta` field of the chunk, and `result += ...` appends this partial content to the overall result.
   
3. **Building the Response**:
   - As the streaming response progresses, each new `delta.content` is added to `result`, which gradually constructs the full response from the model. By doing this in real-time, users can see the response being generated incrementally.

### In the Code:
```python
result += chunk.choices[0].delta.content or ""
```
- This line adds the new content from the current chunk (`chunk.choices[0].delta.content`) to the `result`.
- The `or ""` part ensures that if `delta.content` is `None` or empty in any chunk, it doesn’t cause an error.

### Summary:
- **Delta** refers to the new, incremental part of the response generated by the model in streaming mode.
- The code is appending these incremental updates (deltas) to `result`, allowing the response to be streamed back to the user in real time.

### Stream GPT

In [31]:
# Let's create a call that streams back results
system_message = "You are a helpful assistant with an Australian accent and sunny demeanor"

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [36]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e4ca70b4fd17defb8f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7864 <> https://d358016b146ab362a2.gradio.live
Killing tunnel 127.0.0.1:7865 <> https://d80ca0f2db48318394.gradio.live
Killing tunnel 127.0.0.1:7866 <> https://e4ca70b4fd17defb8f.gradio.live


### Stream Claude

When working with the ChatGPT API (from OpenAI) and the Claude API (from Anthropic), there are a few key differences in how you interact with them. Let’s break down the differences using your code examples:

### 1. **API Structure and Method Calls**:
   - **ChatGPT (OpenAI)**:
     - You are using `openai.chat.completions.create()` to send a prompt and get the response.
     - The API call is structured around the roles of **system** and **user**, and you pass a list of message objects (with roles like `"system"` and `"user"`).
     - **Streaming**: The `stream=True` parameter allows the API to return the response in chunks (delta updates), which you handle in a loop.

   - **Claude (Anthropic)**:
     - You are using `claude.messages.stream()` to send a prompt and stream the response.
     - The Claude API also supports roles, but in your case, the system message is passed as a separate parameter (`system=system_message`), and the messages only include the user’s message.
     - **Streaming**: Claude’s streaming response is accessed via `stream.text_stream`, and chunks of text are processed similarly to how ChatGPT works, but through a `with result as stream` block, which handles the stream lifecycle.

### 2. **System Messages**:
   - **ChatGPT**: The system message is included as part of the `messages` list with the role `"system"`. This defines the assistant’s behavior, tone, or persona (in your example, an Australian accent and sunny demeanor).
   - **Claude**: The system message is passed separately via the `system` parameter, rather than being embedded in the messages array.

### 3. **Streaming the Response**:
   - **ChatGPT**:
     - You iterate over `stream` directly, and each chunk contains partial content from the model’s response in `chunk.choices[0].delta.content`.
     - You construct the result incrementally by appending each chunk of content.
   - **Claude**:
     - You handle streaming by using `with result as stream`, which is more explicit in handling the stream's lifecycle.
     - Inside this block, you access `stream.text_stream`, which is used to yield the response in chunks (similar to ChatGPT’s delta updates).

### 4. **Model Names**:
   - **ChatGPT**: You specify the model via the `model` parameter (e.g., `'gpt-4o-mini'`). OpenAI has different models (GPT-3, GPT-4, etc.), and you select which one to use.
   - **Claude**: You specify the model with the `model` parameter (e.g., `'claude-3-haiku-20240307'`). Anthropic has its own set of models, such as Claude, and different versions might exist for performance variations.

### 5. **Token and Temperature Control**:
   - **ChatGPT**: You haven’t specified it here, but you can control the number of tokens, temperature, and other settings similar to how you would with Claude.
   - **Claude**: In your code, you're specifying `max_tokens=1000` and `temperature=0.7`. These parameters control how long the response can be (in tokens) and how creative or random the model's output should be (temperature).

### Summary of Key Differences:
- **System Message Handling**: ChatGPT uses the system message inside the `messages` list, while Claude uses a separate `system` parameter.
- **Streaming**: ChatGPT streams chunks using `stream=True` with direct iteration, while Claude uses `with result as stream` to handle text streaming.
- **API Calls**: The APIs use slightly different method calls (`openai.chat.completions.create()` vs. `claude.messages.stream()`), but both are designed to process and stream large responses in chunks.
- **Model Naming**: Each API has its own model names and versioning system.
- **Token and Temperature Control**: Both allow for controlling token length and output randomness, though you’ve only specified this in the Claude call.

### What You Should Focus On:
- **Streaming Behavior**: Understand how both APIs stream data in real-time and how the delta or text chunks are processed.
- **System Message and Prompt Structure**: Both ChatGPT and Claude allow you to control the assistant’s behavior via system messages, but they handle this differently.
- **API-Specific Syntax**: Each API has its own method and parameter requirements, so knowing the syntax and available options for each is important when switching between them.

In [ ]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    for chunk in result:
        yield chunk

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never"
)
view.launch()

## Build a Brochure Generator

- start with out Website class to extract links and important content

In [38]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [39]:
system_prompt = "You are a helpful assistant that analyzes the contents of a company website landing page"
system_prompt += "You use the website landing page contents to create a short brochure about the company \
                  for prospective customers, investors and recruits."
system_prompt += "Respond in markdown."

# define brochuer function
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    for chunk in result:
        yield chunk

In [40]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL:"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    allow_flagging="never"
)
view.launch(share=True, debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8ed339152c30647f0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7866 <> https://8ed339152c30647f0c.gradio.live
